# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from pprint import pprint
import pathlib
from tqdm.notebook import tqdm

output_data_file = "output_data/cities.csv"

# Import API key
from api_keys import g_key

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Set file path for the csv file
filepath = pathlib.Path(output_data_file)
# Read csv file
original_df = pd.read_csv(filepath)
original_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ancud,-41.87,-73.82,284.82,93,75,7.70,CL,1589057451
1,pangnirtung,66.15,-65.71,272.97,92,100,2.65,CA,1589057692
2,provideniya,64.38,-173.30,277.12,76,100,11.46,RU,1589057692
3,praya,14.92,-23.51,296.15,73,75,8.20,CV,1589057417
4,solnechnyy,50.72,136.63,281.84,59,100,1.11,RU,1589057693


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Create new dataframe weather_df
weather_df = original_df.copy()
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[['Lat', 'Lng']].astype(float)
# Drop na if there is any
locations.dropna()
# Store the humidity into humidity_num
humidity_num = weather_df['Humidity'].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure(center=(34, -118), zoom_level=1.2)

# humidity_layer = gmaps.heatmap_layer(locations, weights=humidity_num)
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity_num, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)
# Add the layer to figure
fig.add_layer(humidity_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create weather criteria, set max temp, min temp, wind speed, cloudiness
max_temp = (80 + 459.67)*5/9
min_temp = (70 + 459.67)*5/9
max_wind_speed = 10
cloudiness = 0
# Set boolean for each weather criteria
temp_boolean = ((weather_df['Max Temp'] >= min_temp) & (weather_df['Max Temp'] < max_temp))
wind_speed_boolean = (weather_df['Wind Speed'] <= max_wind_speed)
cloudiness_boolean = (weather_df['Cloudiness'] == cloudiness)
# Create a new filtered dataframe passing in the booleans
weather_filtered_df = weather_df.loc[(temp_boolean & wind_speed_boolean & cloudiness_boolean),:]
weather_filtered_df = weather_filtered_df.reset_index(drop=True)
weather_filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,arraial do cabo,-22.97,-42.03,295.15,68,0,2.60,BR,1589057707
1,kutum,14.20,24.67,295.36,16,0,3.87,SD,1589057746
2,tartagal,-22.52,-63.80,296.28,58,0,1.13,AR,1589057751
3,safaga,26.73,33.94,298.15,41,0,2.10,EG,1589057482
4,doctor juan leon mallorquin,-25.40,-55.27,298.15,41,0,6.20,PY,1589057776
5,aswan,24.09,32.91,297.26,23,0,6.27,EG,1589057644
6,guerrero negro,27.98,-114.06,298.29,44,0,5.91,MX,1589057799
7,tura,25.52,90.22,296.72,41,0,1.49,IN,1589057805
8,srivardhan,18.03,73.02,299.02,85,0,2.21,IN,1589057825
9,walvis bay,-22.96,14.51,296.49,38,0,2.13,NaN,1589057852


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create new dataframe for hotel
hotel_df = pd.DataFrame()
# Search hotel in google place
target_search = 'Hotel'
# Put lat and lng from the weather_filtered_df to list
lat = weather_filtered_df['Lat']
lng = weather_filtered_df['Lng']
# Loop through the lat and lng and using google API to search the hotel
for i in tqdm(range(len(weather_filtered_df['City']))):
    # redefine params
    params = {
        "location": f'{lat[i]},{lng[i]}',
        "keyword": target_search,
        "radius": 50000,
        "key": g_key
    }
    # Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Pull hotel data from API
    hotel = requests.get(base_url, params).json()
    # Append the hotel data to hotel_df
    hotel_df = hotel_df.append([[
        hotel['results'][0]['name'],
        hotel['results'][0]['geometry']['location']['lat'],
        hotel['results'][0]['geometry']['location']['lng'],
        weather_filtered_df['City'][i],
        weather_filtered_df['Country'][i]
    ]])

In [6]:
hotel_df.columns=['Hotel Name', 'Lat', 'Lng', 'City', 'Country']
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Orlanova Hotel,-22.961411,-42.021496,arraial do cabo,BR
1,مسجد دليبة العتيق,14.355573,24.491177,kutum,SD
2,Hotel Pórtico Norte,-22.541943,-63.803578,tartagal,AR
3,SENTIDO Mamlouk Palace,27.099844,33.828067,safaga,EG
4,Cide Resort Hotel,-25.487180,-54.782938,doctor juan leon mallorquin,PY
5,Sofitel Legend Old Cataract Aswan,24.094242,32.897351,aswan,EG
6,Hotel Los Caracoles,27.967758,-114.028336,guerrero negro,MX
7,Hotel Polo Orchid,25.514468,90.192206,tura,IN
8,The Fern Samali Resort,17.739743,73.165860,srivardhan,IN
9,Strand Hotel Swakopmund,-22.675519,14.521949,walvis bay,NaN


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(34, -118), zoom_level=1.5)

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_layer)
# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Create a combined map
fig = gmaps.figure(center=(34, -118), zoom_level=1.2)

fig.add_layer(humidity_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))